### NLP pre-processing

Import and load data

In [2]:
import pandas as pd
import unicodedata
import contractions
import re
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
df_train = pd.read_csv('train.csv', names=['rating','title','text'])
df_test= pd.read_csv('test.csv', names=['rating','title','text'])

In [6]:
df_train.head()

,rating,title,text
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


In [10]:
df_train = df_train[['rating','text']]
df_test = df_test[['rating','text']]

Changed the rating from 0 to 5 to good, bad or neutral. Not sure if it was the best approach though

In [18]:
def good_bad_neutral(number):
    if number > 3:
        return 'good'
    elif number == 3:
        return 'neutral'
    else:
        return 'bad'

In [ ]:
pattern = r'[^a-zA-Z\s]'
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

Removing accented words, contractions, special characters, multiple whitespaces from text and making it lowercase

In [13]:
df_train.text = df_train.text.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df_train.text = df_train.text.apply(lambda x: contractions.fix(x))
df_train.text = df_train.text.apply(lambda x:re.sub(pattern, '', x))
df_train.text = df_train.text.apply(lambda x: x.lower())
df_train.text = df_train.text.apply(lambda x: strip_multiple_whitespaces(x))

In [19]:
df_train.rating = df_train.rating.apply(lambda x: good_bad_neutral(x))
df_train.text = df_train.text.apply(lambda x: word_tokenize(x))
df_train.text = df_train.text.apply(lambda x: [word for word in x if word not in stop_words])
df_train.text = df_train.text.apply(lambda x: [porter.stem(word) for word in x])

In [50]:
df_train.head()

,rating,text
0,good,"[hope, lot, peopl, hear, cd, need, strong, pos..."
1,good,"[read, lot, review, say, best, game, soundtrac..."
2,good,"[music, yasunori, misuda, without, question, c..."
3,good,"[probabl, greatest, soundtrack, histori, usual..."
4,good,"[reason, cd, expens, even, version, importsom,..."


I'm not sure if it necessary to join the document or having a list of the words is enough

In [51]:
df_train_joined = df_train
df_train_joined.text = df_train_joined.text.apply(lambda x: ' '.join(x))
df_train_joined.head()

,rating,text
0,good,hope lot peopl hear cd need strong posit vibe ...
1,good,read lot review say best game soundtrack figur...
2,good,music yasunori misuda without question close s...
3,good,probabl greatest soundtrack histori usual bett...
4,good,reason cd expens even version importsom best m...


In [60]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_joined.text)

In [36]:
df_test.text = df_test.text.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df_test.text = df_test.text.apply(lambda x: contractions.fix(x))
pattern = r'[^a-zA-Z\s]'
df_test.text = df_test.text.apply(lambda x:re.sub(pattern, '', x))
df_test.text = df_test.text.apply(lambda x: x.lower())
df_test.text = df_test.text.apply(lambda x: strip_multiple_whitespaces(x))

In [37]:
df_test.rating = df_test.rating.apply(lambda x: good_bad_neutral(x))
df_test.text = df_test.text.apply(lambda x: word_tokenize(x))
df_test.text = df_test.text.apply(lambda x: [word for word in x if word not in stop_words])
df_test.text = df_test.text.apply(lambda x: [porter.stem(word) for word in x])

In [61]:
df_test_joined = df_test
df_test_joined.text = df_test_joined.text.apply(lambda x: ' '.join(x))
df_test_joined.head()

,rating,text
0,good,fast read fill unexpect humour profound insigh...
1,bad,bought one chargersth instruct say light stay ...
2,bad,excit find book ostens muslim femin volum live...
3,bad,big jvc fan like model suspisci saw sever unit...
4,bad,love style coupl year dvd give problem even wo...


In [62]:
X_test = vectorizer.fit_transform(df_test_joined.text)